# Run the R Shiny demo

In this notebook, we will run an R Shiny application that will be 
exposed in a route for a live map of ecological forecasting sites. 

## Install PostgreSQL R packages

Run the R code below to install the RPostgreSQL and RPostgres R packages locally. 

In [ ]:
install.packages("RPostgreSQL", lib="/opt/app-root/src/Rpackages")
install.packages("RPostgres", lib="/opt/app-root/src/Rpackages")


## Load Postgres connection environment variables

Run the R code below to load the Postgres host, port, user, and password 
from environment variables into R variables. 

In [ ]:
db <- "postgres"
db_host <- Sys.getenv("PGHOST")
db_port <- Sys.getenv("PGPORT")
db_user <- Sys.getenv("PGUSER")
db_pass <- Sys.getenv("PGPASSWORD")
print("DONE")

## Load Shiny UI libraries for ecological forecasting site map

Load the R libraries to run the application, like `shiny`, `leaflet` for maps, `DBI`, `RPostgreSQL`, and `RPostgres` for the connection to the PEcAn database. 

In [ ]:
library(shiny)
library(leaflet)
library(DBI)
library(RPostgreSQL)
library(RPostgres)

ui <- fluidPage(
  tags$h1("Ecological Forecasting Sites"),
  leafletOutput(outputId = "map")
)
print("DONE")

## Create the Shiny server

Create the server callback function that includes: 

- `conn`: a reactive data connection to PostgreSQL. 
- `sites`: The sites returned from the SQL query to PostgreSQL. 
- disconnect from PostgreSQL. 
- frame the sites data. 
- Create an output map with `renderLeaflet`. 
- The `sitename` database field will be the label. 
- The `lat` and `lon` database fields will be the map coordinates. 


In [ ]:
server <- function(input, output) {
  data <- reactive({
    conn <- dbConnect(
      RPostgres::Postgres(),
      dbname = "bety",
      host = db_host,
      port = db_port,
      user = db_user,
      password = db_pass
    )
    sites <- dbGetQuery(conn, "SELECT sites.id, sites.sitename, sites.city, sites.country, ST_X(ST_CENTROID(sites.geometry)) AS lon, ST_Y(ST_CENTROID(sites.geometry)) AS lat FROM sites INNER JOIN sitegroups_sites ON sitegroups_sites.site_id=sites.id where sitegroups_sites.sitegroup_id=1")
    dbDisconnect(conn)
    data.frame(sites)
  })

  output$map <- renderLeaflet({
    leaflet(data = data()) %>%
      addTiles() %>%
      addMarkers(~lon, ~lat, label = ~sitename) %>%
      addProviderTiles(providers$Esri.WorldStreetMap)
  })
}
print("DONE")

## Run the R Shiny app in development

You can run the Shiny app by running the commands below. 
Keep in mind that this is a blocking command for the Jupyter 
Notebook, so you won't be able to run other commands in this 
Notebook until you interrupt the command. See below for details. 

In [ ]:
app <- shinyApp(ui = ui, server)
shiny::runApp(host = '0.0.0.0', port=8080, app=app)



## Visit your RShiny app in development

Click on the link that you generated at the very end of the [03-prepare-rshiny-demo.ipynb](03-prepare-rshiny-demo.ipynb) notebook to access the RShiny app in development.

[![R Shiny ecological forecasting sites app](pictures/rshiny-app-eco-forecast-sites.png)](rshiny-app-eco-forecast-sites.png)


## Stopping the RShiny app

When you are done running the R Shiny app, you will want to stop 
the notebook code you ran above by clicking the "Interrupt the kernel" 
button, or press [ i ] [ i ] twice in a row in this notebook to 
interrupt the kernel. 
